# Simulated annealing

The annealing is a metallurgical process in which the temperature of a metal is increased and then gradually decreased. In this way, the molecules of the material are first brought into a highly disordered state and then they move to a more ordered disposition i.e. the crystal lattice. The cooling must occur sufficently slowly but then the material will result hardened by the process. A similar approach can be used to solve optimization processes which require to find the minimum (or the maximum) of a multi-dimentionate function, in presence of multiple local minima (or maxima), which make it impossible to use exact methods such as gradient descent. In opposition to Metropolis-Hastings method, which always samples configurations with lower energy than the current one, the simulated annealing can select configurations whose energy is higher than the current one with a certain probability. This probability is very high in the first stages of the algorithm, when the temperature is high, then, as the cooling moves takes place,  it will be more and more unlikely to select energetically-disadvantageous configurations. This part of the algorithm is fundamental because it allows us to avoid local minima or maxima, which act as traps for any exact algorithm. The cooling must occur slowly enough to allow the algorithm to reach the global minimum of the function. 

A common example of usage of this algorithm is to solve the problem of the travelling salesman. By the way, it finds its main applications in computational chemistry, digital electronics, jobs placement and, in general, in all those contexts where the research space is discrete.

As a first approach to this algorithm, we try to solve the travelling salesman problem. In first place we want to generate a set of random points constituting the cities to be visited by the salesman. 

In [ ]:
import random
from math import exp
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def get_random_points(N_points, x_max, y_max):
    x_values = [random.randint(0,x_max) for i in range(N_points)]
    y_values = [random.randint(0,y_max) for i in range(N_points)]

    # append to the lists their initial elements in order to close the lines when points are plotted

    x_values.append(x_values[0])
    y_values.append(y_values[0])

    route = [[i,j] for i,j in zip(x_values,y_values)]

    #print(route)

    return route

Now we have to generate a path between all these points. The first path has to be randomly-generated. Then, we have to compute the length of the path. The total length of the path represents the cost function which has to be minimized by the algorithm.

In [ ]:
def distance_two_points(x1, x2, y1, y2):
    distance = sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

In [ ]:
def total_distance(route): # this is the objective function in the travelling slesman problem

    total_distance = 0

    for point_index, point in enumerate(route[:-1]):
        total_distance = total_distance + distance_two_points(point[0], route[point_index+1][0], point[1], route[point_index+1][1])

    return total_distance

In [ ]:
def new_delimiter(delimiter_old, delimiter_min, reset_value, P):

    delimiter_new = delimiter_old*0.9999

    if delimiter_new < delimiter_min:
        delimiter_new = reset_value

    return delimiter_new

In the simulated annealing algorithm, we are asked to generate new routes. Hence we need a function that receives the list of points to be visited and shuffles them in such a way that they will be visited in a different order i.e. a different route. We shuffle all the points except the first and the last one in order to close the line whatever is the route. We stress the fact that the points in the route are always the same but they are listed in the order in which they are going to be visited.

In [ ]:
# extract two elements to be swapped and check whether they respect the delimiter
# the function also returns the two points involved in the swap for reasons related to the delimiter comparison

def get_new_route(route, delimiter, P):

    shuffled_route = list.copy(route)
    first_point, last_point, middle_points = shuffled_route[0], shuffled_route[-1], shuffled_route[1:-1]

    # extract two elements to be swapped

    position1 = random.randint(0,len(middle_points)-1)
    position2 = random.randint(0,len(middle_points)-1)

    #get and swap the two points

    middle_points[position1], middle_points[position2] = middle_points[position2], middle_points[position1]

    shuffled_route = [first_point] + middle_points + [last_point]
    return shuffled_route, middle_points[position1], middle_points[position2]

We need a function to plot the points in a given route.

In [ ]:
def print_route(route):
    xs = [point[0] for point in route]
    ys = [point[1] for point in route]
    #plt.plot(xs, ys, 'bo', linestyle='--')
    return xs, ys

At this point we can start working on the algorithm itself. In first place we need a function implementing the cooling schedule i.e. producing decreasing temperature values at a certain rate. We choose the cooling schedule:

$$T_{i+1} = \frac{T_i}{\frac{i}{\alpha} + 1} \hspace{1cm} \alpha \in \mathcal{R}$$

As a second and more robust cooling procedure we can try to implement the Lam schedule, which mimics the behaviour observed in many, long-lasting successful experiments.

In [ ]:
def Lam_schedule(P, T):
    if P > 0.96:
        T *= 0.5
        return T 
    if (P <= 0.95 and P > 0.8):
        T *= 0.9
        return T 
    if (P <= 0.8 and P > 0.15):
        T *= 0.95
        return T
    if (P <= 0.15):
        T *= 0.8
        return T

In [ ]:
# this function keeps making swaps until the produced one respects the delimiter constraint

def validate_swap(current_route, delimiter, P, delimiter_min, reset_value):
    is_valid = False
    new_route = current_route  # Initialize new_route outside the loop

    while not is_valid:
        new_route, swapped_point_1, swapped_point_2 = get_new_route(new_route, delimiter, P)
        x1, x2, y1, y2 = swapped_point_1[0], swapped_point_2[0], swapped_point_1[1], swapped_point_2[1]
        if distance_two_points(x1, x2, y1, y2) <= delimiter:
            is_valid = True

    delimiter = new_delimiter(delimiter, delimiter_min, reset_value, P)  # Update delimiter

    return delimiter, new_route  # Return the updated delimiter and new_route

In [ ]:
def simulated_annealing(initial_points, T0, Tmin, delimiter0):

    current_route = initial_points
    current_length = total_distance(current_route)
    T = T0
    iteration = 0
    acceptance_prob = 0.99
    delimiter = delimiter0

    while T > Tmin:
        r = np.random.random()

        # here "validate swap" indicates that the new configuration has to respect the limit imposed by the swap delimiter

        delimiter, new_route = validate_swap(current_route, delimiter, acceptance_prob, 0.5, 10)
        distance_diff = total_distance(new_route) - total_distance(current_route)
        acceptance_prob = np.exp(-distance_diff / T)

        if distance_diff <= 0 or acceptance_prob > r:
            current_route = new_route
            print(f"Iteration: {iteration}, Temperature: {T}, Distance: {total_distance(current_route)}, Delimiter: {delimiter}, Route: {current_route}")
            #T = Lam_schedule(T, acceptance_prob)  # Update temperature with Lam schedule
            T = 0.999*T

            iteration += 1

    final_distance = total_distance(current_route)
    return current_route, final_distance, iteration, T

Here we propose a function for low-temperature annealing, which can be invoked to optimize a sub-optimal result obtained by a high-temperature simulated annealing. The function receives as input the sub-optimal route as well as the exponent of the initial and final temperatures. Then, the sub-optimal route is processed via simualted annealing with the given data. If the length of the resulting route is smaller, then restart the annealing by decreasing by 1 the exponent of the initial temperature. This is needed to keep optimizing the route without disrupting the ordered structure obtained in previous annealing schedules. If the length is intstead longer then the previous one, then it is admissible to increase by 1 the exponent, in order to move away fro mthe local minimum. We keep doing so until the exponent of the initial temperature is equal to - 2 (I played with the algorithm many times and I found that there are no more swaps once the temperateure reached $10^{-2}$ but I remain open to more rigorous formulations).

In [ ]:
def low_T_annealing(sub_optimal_route, initial_exponent, final_exponent):

    try:
        route_to_process = sub_optimal_route
        distance = total_distance(route_to_process)
        exponent = initial_exponent
        last_exponent = final_exponent
        delimiter = 84

        #lists containing the best route and length at each iteration
        #this is required because the algorithm may find an optimal solution but it could lose it
        #if it has to be run again

        best_configurations = []
        best_lengths = []

        while exponent > 0:
            current_route, final_distance, iteration, T = simulated_annealing(route_to_process, 10**(exponent), 10**(last_exponent), delimiter)

            if final_distance < distance: # a better configuration has been found

                #decrease the next exponent to avoid excessive re-shuffling
                exponent = exponent - 1
                #delimiter = delimiter - 5
                route_to_process = current_route

                # add the new best configuration to the list of best configurations found
                best_configurations.append(route_to_process)
                best_lengths.append(final_distance)

            else:
                # this "if" is needed to avoid that the exponent increases too much and disrupts the ordered configuration
                # in addition to making the algorithm run excessively long
                if exponent < 2:
                  #increase the next exponent to add some re-shuffling
                  exponent = exponent + 1
                  #delimiter = delimiter + 4

        optimal_route = current_route
        optimal_distance = total_distance(optimal_route)

        # in the first case, the optimum solution is exactly the last one
        if optimal_distance == min(best_lengths):
            return optimal_route, optimal_distance

            # otherwise, it was found but discarded because the stopping condition of the algorithm was still to be met
        else:
            min_index = best_lengths.index(min(best_lengths))
            optimal_route = best_configurations[min_index]
            optimal_distance = best_lengths[min_index]
            return optimal_route , optimal_distance

        # if the algorithm lasts too long the user can stop it
        # the program will return the best cofiguration it was able to find up to the interrupt
    except KeyboardInterrupt:
        print("Low temperature annealing stopped by the user")
    finally:

        # if the algorithm effectively found some better configuration, provide it to the user

        if len(best_lengths) > 0:
            # return the best configuration found
            min_index = best_lengths.index(min(best_lengths))
            optimal_route = best_configurations[min_index]
            optimal_distance = best_lengths[min_index]
            return optimal_route , optimal_distance

        else:
            print("The low-temperature annealing did not find any possible improvement.")


In [ ]:
# 60 points means 59! possible routes, which is in the same order of magnitude of Eddington number i.e. the estimate of protons in the universe.
N_points = 60
x_max = 60
y_max = 60

route0 = get_random_points(N_points, x_max, y_max)
print(f"Initial route: {route0}")
print(f"Length of initial route: {total_distance(route0)}")
xs, ys = print_route(route0)
plt.plot(xs, ys, 'bo', linestyle='--')
plt.grid(True)
plt.show

In [ ]:
final_route, final_distance, final_iteration, final_temperature = simulated_annealing(route0, 10**(10), 10**(-2), 84)

#print(f"Final distance: {final_distance}")
print(final_route)
xs, ys = print_route(final_route)
plt.title('Temperature= {}'.format(format(final_temperature, '.2e')) + '  ' + 'Iteration= {}'.format(final_iteration) + '  ' + 'Distance= {}'.format(round(final_distance), 2))
plt.plot(xs, ys, 'bo', linestyle='--')
plt.grid(True)
plt.show

If the first simulated annealing did not produce the optimal solution we can run it again strating from a low temperature in order to keep the optimized path as much as possible.

In [ ]:
optimal_route, optimal_distance = low_T_annealing(final_route, 2, -2)

xs, ys = print_route(optimal_route)
print("")
print(f"Optimal route: {optimal_route}")
print("")
print(f"Optimal distance: {optimal_distance}")
#plt.title('Distance= {}'.format(round(optimal_distance), 2) + ' ')
plt.plot(xs, ys, 'bo', linestyle='--')
plt.grid(True)
plt.show